# Package Installation

In [1]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 113.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 17.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 5.0.0 which is incompatible.
gradio 5.49.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.5 which is incompatible.


In [2]:
import pandas as pd
import sacrebleu
import numpy as np
import torch
import transformers
import peft
import datasets
from datasets import Dataset
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer, 
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer, 
    Seq2SeqTrainingArguments
)
from peft import LoraConfig, get_peft_model, TaskType

print("torch", torch.__version__)
print("transformers", transformers.__version__, "| peft", peft.__version__)
print("datasets", datasets.__version__)
print("GPU count:", torch.cuda.device_count())

torch 2.8.0+cu126
transformers 5.0.0 | peft 0.18.1
datasets 4.5.0
GPU count: 2


In [ ]:
from huggingface_hub import login
hf_token = ""
login(token=hf_token)

# Load data

In [4]:
def load_txt(path):
    with open(path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f.readlines()]

train_en = load_txt('/kaggle/input/vlsp-dataset/data/train.en.txt')
train_vi = load_txt('/kaggle/input/vlsp-dataset/data/train.vi.txt')
test_en = load_txt('/kaggle/input/vlsp-dataset/data/public_test.en.txt')
test_vi = load_txt('/kaggle/input/vlsp-dataset/data/public_test.vi.txt')

df_train = pd.DataFrame({'en': train_en, 'vi': train_vi}).drop_duplicates().reset_index(drop=True)
df_test = pd.DataFrame({'en': test_en, 'vi': test_vi}).reset_index(drop=True)

train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

print(f"Train size: {len(train_dataset)}")
print(f"Test size: {len(test_dataset)}")

Train size: 348224
Test size: 3000


In [5]:
example_en = df_train.iloc[0]['en']
example_vi = df_train.iloc[0]['vi']
print(example_en, example_vi, sep='\n\n')

To evaluate clinical, subclinical symptoms of patients with otitis media with effusion and V.a at otorhinolaryngology department – Thai Nguyen national hospital

Nghiên cứu đặc điểm lâm sàng, cận lâm sàng bệnh nhân viêm tai ứ dịch trên viêm V.A tại Khoa Tai mũi họng - Bệnh viện Trung ương Thái Nguyên


# Prepare model

In [6]:
MODEL_NAME = "vinai/vinai-translate-en2vi-v2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, src_lang="en_XX", tgt_lang="vi_VN")
print(tokenizer.pad_token)
# if tokenizer.pad_token is None:
#     tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"
model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

model.config.use_cache = False

config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.41M [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


<pad>


pytorch_model.bin:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/519 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/accelerate/utils/modeling.py:1598: UserWarning: The following device_map keys do not match any submodules in the model: ['final_logits_bias']
  warnings.warn(


In [7]:
vi_token_id = tokenizer.lang_code_to_id["vi_VN"]
model.generation_config.decoder_start_token_id = vi_token_id
model.generation_config.forced_bos_token_id = vi_token_id

In [8]:
import re
def get_num_layers(model):
    numbers = set()
    for name, _ in model.named_parameters():
        for number in re.findall(r'\d+', name):
            numbers.add(int(number))
    return max(numbers)

def get_last_layer_linears(model):
    names = []
    
    num_layers = get_num_layers(model)
    for name, module in model.named_modules():
        if str(num_layers) in name and not "encoder" in name:
            if isinstance(module, torch.nn.Linear):
                names.append(name)
    return names

In [9]:
get_num_layers(model)

11

In [10]:
get_last_layer_linears(model)

['model.decoder.layers.11.self_attn.k_proj',
 'model.decoder.layers.11.self_attn.v_proj',
 'model.decoder.layers.11.self_attn.q_proj',
 'model.decoder.layers.11.self_attn.out_proj',
 'model.decoder.layers.11.fc1',
 'model.decoder.layers.11.fc2']

In [11]:
model

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(66773, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(66773, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)


# LoRA config

In [12]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj", "fc1", "fc2"]
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 8,650,752 || all params: 636,977,152 || trainable%: 1.3581


In [13]:
model

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): MBartForConditionalGeneration(
      (model): MBartModel(
        (shared): MBartScaledWordEmbedding(66773, 1024, padding_idx=1)
        (encoder): MBartEncoder(
          (embed_tokens): MBartScaledWordEmbedding(66773, 1024, padding_idx=1)
          (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
          (layers): ModuleList(
            (0-11): 12 x MBartEncoderLayer(
              (self_attn): MBartAttention(
                (k_proj): lora.Linear(
                  (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1024, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=16, out_feature

# Generate example

In [14]:
example_en = df_train.iloc[0]['en']
example_vi = df_train.iloc[0]['vi']
print(example_en, example_vi, sep='\n\n')

To evaluate clinical, subclinical symptoms of patients with otitis media with effusion and V.a at otorhinolaryngology department – Thai Nguyen national hospital

Nghiên cứu đặc điểm lâm sàng, cận lâm sàng bệnh nhân viêm tai ứ dịch trên viêm V.A tại Khoa Tai mũi họng - Bệnh viện Trung ương Thái Nguyên


In [15]:
import torch

sentence = example_en
inputs = tokenizer(sentence, return_tensors="pt").to(model.device)
vi_token_id = tokenizer.lang_code_to_id["vi_VN"]

print("Đang dịch...")

translated_tokens = model.generate(
    **inputs,
    max_length=512,
    forced_bos_token_id=vi_token_id, 
    num_beams=5,
    early_stopping=True
)

output_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

print("-" * 30)
print(f"Input: {sentence}")
print(f"Output: {output_text}")

Đang dịch...
------------------------------
Input: To evaluate clinical, subclinical symptoms of patients with otitis media with effusion and V.a at otorhinolaryngology department – Thai Nguyen national hospital
Output: Đánh giá đặc điểm lâm sàng, cận lâm sàng của bệnh nhân viêm tai giữa tràn dịch màng phổi và viêm tai giữa mạn tính tại khoa Tai Mũi Họng – Bệnh viện Trung ương Thái Nguyên


# Format finetune data

In [16]:
def preprocess_function(examples):
    inputs = [ex for ex in examples["en"]]
    targets = [ex for ex in examples["vi"]]
    
    model_inputs = tokenizer(
        inputs, 
        max_length=512, 
        truncation=True, 
        padding=False 
    )
    
    labels = tokenizer(
        text_target=targets, 
        max_length=512, 
        truncation=True, 
        padding=False
    )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_test = test_dataset.map(preprocess_function, batched=True, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/348224 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [17]:
print(tokenized_train[0].keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [18]:
print(tokenized_train[0])

{'input_ids': [490, 4507, 41, 3114, 167, 4, 1443, 9579, 1473, 3344, 8, 1078, 40, 1585, 16986, 7, 2987, 40, 9630, 7239, 9, 424, 5, 86, 103, 1585, 1199, 7581, 1923, 1174, 108, 2195, 3793, 428, 7499, 7199, 1628, 2449, 2, 66750], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [2865, 452, 468, 300, 2004, 1288, 4, 2085, 2004, 1288, 277, 129, 1854, 1791, 13, 1784, 219, 94, 1854, 424, 5, 617, 100, 3414, 7260, 3013, 172, 108, 20, 2414, 920, 342, 2267, 2736, 3831, 2, 66770]}


# Train

In [19]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    
    result = sacrebleu.corpus_bleu(decoded_preds, decoded_labels)
    return {"bleu": result.score}

In [20]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    padding=True,
    pad_to_multiple_of=8
)

In [21]:
targs = Seq2SeqTrainingArguments(
    output_dir="./finetuned_model",
    fp16=True,       
    bf16=False,
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2, 
    learning_rate=5e-5,
    num_train_epochs=1,    
    logging_steps=250,
    save_strategy="epoch", #steps/epoch
    eval_strategy="epoch",
    # save_steps=250,        
    save_total_limit=2,
    report_to="none",
    group_by_length=True, 
    max_grad_norm=1.0,
    ddp_find_unused_parameters=False,
    dataloader_num_workers=4,
    remove_unused_columns=False,
    predict_with_generate=True,
    generation_config=model.generation_config,
    generation_max_length=512
)
model.generation_config.decoder_start_token_id = vi_token_id
model.generation_config.forced_bos_token_id = vi_token_id

In [22]:
trainer = Seq2SeqTrainer(
    model=model,
    args=targs,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [23]:
print("Start training...")
trainer.train()

Start training...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epoch,Training Loss,Validation Loss,Bleu
1,1.699509,0.918506,63.372431


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=10882, training_loss=1.7395522165464974, metrics={'train_runtime': 6159.1413, 'train_samples_per_second': 56.538, 'train_steps_per_second': 1.767, 'total_flos': 3.828682119551386e+16, 'train_loss': 1.7395522165464974, 'epoch': 1.0})

In [24]:
for name, param in model.named_parameters():
    if "lora_A" in name:
        print(name, param.data.abs().mean().item())
        break

base_model.model.model.encoder.layers.0.self_attn.k_proj.lora_A.default.weight 0.016382616013288498


In [25]:
import torch

sentence = example_en
inputs = tokenizer(sentence, return_tensors="pt").to(model.device)
vi_token_id = tokenizer.lang_code_to_id["vi_VN"]

print("Đang dịch...")

translated_tokens = model.generate(
    **inputs,
    max_length=512,
    forced_bos_token_id=vi_token_id, 
    num_beams=5,
    early_stopping=True
)

output_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

print("-" * 30)
print(f"Input: {sentence}")
print(f"Output: {output_text}")

Đang dịch...
------------------------------
Input: To evaluate clinical, subclinical symptoms of patients with otitis media with effusion and V.a at otorhinolaryngology department – Thai Nguyen national hospital
Output: Đánh giá đặc điểm lâm sàng, cận lâm sàng của bệnh nhân viêm tai giữa tràn dịch và viêm tai giữa mạn tính tại khoa Tai Mũi Họng – Bệnh viện Trung ương Thái Nguyên


# Save model

In [26]:
trainer.save_model("vinai-en2vi-v2-final")
tokenizer.save_pretrained("vinai-en2vi-v2-final")

('vinai-en2vi-v2-final/tokenizer_config.json',
 'vinai-en2vi-v2-final/tokenizer.json')

# Loading and using the model later

In [27]:
import torch
import gc
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel

In [28]:
try:
    del model
    del base_model
    del trainer
except NameError:
    pass

In [29]:
gc.collect()
torch.cuda.empty_cache()

In [30]:
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    "vinai/vinai-translate-en2vi-v2",
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

Loading weights:   0%|          | 0/519 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
/usr/local/lib/python3.12/dist-packages/accelerate/utils/modeling.py:1598: UserWarning: The following device_map keys do not match any submodules in the model: ['final_logits_bias']
  w

In [31]:
peft_model_id = "vinai-en2vi-v2-final"

In [32]:
tokenizer = AutoTokenizer.from_pretrained(peft_model_id, src_lang="en_XX", tgt_lang="vi_VN")
model = PeftModel.from_pretrained(base_model, peft_model_id)

In [33]:
vi_token_id = tokenizer.lang_code_to_id["vi_VN"]
model.generation_config.decoder_start_token_id = vi_token_id
model.generation_config.forced_bos_token_id = vi_token_id

In [34]:
model.eval()

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): MBartForConditionalGeneration(
      (model): MBartModel(
        (shared): MBartScaledWordEmbedding(66773, 1024, padding_idx=1)
        (encoder): MBartEncoder(
          (embed_tokens): MBartScaledWordEmbedding(66773, 1024, padding_idx=1)
          (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
          (layers): ModuleList(
            (0-11): 12 x MBartEncoderLayer(
              (self_attn): MBartAttention(
                (k_proj): lora.Linear(
                  (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1024, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=16, out_feature

In [35]:
sentence = example_en
inputs = tokenizer(sentence, return_tensors="pt").to(model.device)
vi_token_id = tokenizer.lang_code_to_id["vi_VN"]

print("Đang dịch...")

translated_tokens = model.generate(
    **inputs,
    max_length=512,
    forced_bos_token_id=vi_token_id, 
    num_beams=5,
    early_stopping=True
)

output_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

print("-" * 30)
print(f"Input: {sentence}")
print(f"Output: {output_text}")

Đang dịch...
------------------------------
Input: To evaluate clinical, subclinical symptoms of patients with otitis media with effusion and V.a at otorhinolaryngology department – Thai Nguyen national hospital
Output: Đánh giá đặc điểm lâm sàng, cận lâm sàng của bệnh nhân viêm tai giữa tràn dịch và viêm tai giữa mạn tính tại khoa Tai Mũi Họng – Bệnh viện Trung ương Thái Nguyên
